The objective of this analysis is to get some insights by comparing the spread of the virus with the trend of mobility

# Libraries import

In [17]:
import pandas as pd
import datetime
import numpy as np
import matplotlib
import re
import matplotlib.pyplot as plt
matplotlib.style.use(["ggplot"])
#"https://covid19-static.cdn-apple.com/covid19-mobility-data/2006HotfixDev8/v1/en-us/applemobilitytrends-2020-04-17.csv"

# Importation of the datasets

## Covid-19

The Covid19 dataset is provided by John Hopkins University. The dataset is updated every day

In [18]:
# url towards the Github repository
url_racine = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"

In [19]:
# the date from which CSSE repository started to collect data
date_begin = datetime.date(2020,1,22)

# download of the data
date_end = datetime.date.today()

range_of_date = pd.date_range(date_begin,end=date_end).strftime("%m-%d-%Y").tolist()

In [20]:

range_of_date_iter = range_of_date
iterative = 0
records = pd.DataFrame()
except_dates = []
result = {}
while (len(except_dates)>1 and iterative < 5) or (iterative == 0):
    iterative += 1
    print(except_dates)
    except_dates = []
    for date_given in range_of_date_iter:
        try:
            temp = pd.read_csv(url_racine + date_given + ".csv")
            num = temp.shape[0]
            temp["date"] = np.repeat(date_given,num)
            temp["Date"] = [datetime.datetime.strptime(date_str,'%m-%d-%Y') for date_str in temp.date]
            #temp = temp.drop(columns=["date"])
            records = records.append(temp,sort=False)
        except:
            except_dates.append(date_given)
            pass
    range_of_date_iter = except_dates
    result["table"+str(iterative)]=records
    records = pd.DataFrame()
    
records = result["table1"]

[]


In [21]:
records.dtypes

Province/State            object
Country/Region            object
Last Update               object
Confirmed                float64
Deaths                   float64
Recovered                float64
date                      object
Date              datetime64[ns]
Latitude                 float64
Longitude                float64
FIPS                     float64
Admin2                    object
Province_State            object
Country_Region            object
Last_Update               object
Lat                      float64
Long_                    float64
Active                   float64
Combined_Key              object
dtype: object

we can notice that there are variables with the same information. They are:
* **Province/State** and **Province\_State**
* **Country/Region** and **Country\_Region**
* **Lat** and **Latitude**
* **Long_** and **Longitude**

Let's combine them into one variable each

In [22]:
records["Prov"] = records["Province/State"].combine_first(records["Province_State"])
records["Country"] = records["Country/Region"].combine_first(records["Country_Region"])
records["Lati"] = records["Latitude"].combine_first(records["Lat"])
records["Longi"] = records["Longitude"].combine_first(records["Long_"])

In [23]:
records.drop(columns=["Province/State",
"date",
"Province_State",
"Country_Region",
"Country/Region",
"Last Update",
"Last_Update",
"Longitude",
"Long_",
"Latitude",
"Lat",
"Admin2",
"FIPS",],inplace=True)

In [28]:
print("the shape of the data is: \n{}".format(records.shape))
records.sort_values(["Confirmed"],ascending=False).head()

the shape of the data is: 
(92516, 10)


,Confirmed,Deaths,Recovered,Date,Active,Combined_Key,Prov,Country,Lati,Longi
3025,191726.0,20043.0,74797.0,2020-04-18,96886.0,Spain,NaN,Spain,40.463667,-3.74922
3017,190839.0,20002.0,74797.0,2020-04-17,96040.0,Spain,NaN,Spain,40.463667,-3.74922
3013,184948.0,19315.0,74797.0,2020-04-16,90836.0,Spain,NaN,Spain,40.463667,-3.74922
2998,177644.0,18708.0,70853.0,2020-04-15,88083.0,Spain,NaN,Spain,40.463667,-3.74922
2953,175925.0,23227.0,44927.0,2020-04-18,107771.0,Italy,NaN,Italy,41.871940,12.56738


In [25]:
records.dtypes

Confirmed              float64
Deaths                 float64
Recovered              float64
Date            datetime64[ns]
Active                 float64
Combined_Key            object
Prov                    object
Country                 object
Lati                   float64
Longi                  float64
dtype: object

In [26]:
records.describe()

,Confirmed,Deaths,Recovered,Active,Lati,Longi
count,92497.000000,92075.000000,92128.000000,84899.000000,88712.000000,88712.000000
mean,432.060456,24.204551,106.596561,238.570136,36.221805,-76.410610
std,4980.274773,455.253875,1984.352932,3237.413942,11.210298,48.268481
min,0.000000,0.000000,0.000000,-64840.000000,-51.796300,-170.132000
25%,1.000000,0.000000,0.000000,0.000000,33.588398,-96.012303
50%,6.000000,0.000000,0.000000,0.000000,37.832188,-87.112100
75%,36.000000,1.000000,0.000000,3.000000,41.639360,-79.913511
max,191726.000000,23227.000000,85400.000000,122370.000000,72.000000,178.065000


It seems to have some countries which are not 

In [27]:
records.describe(include="object")

,Combined_Key,Prov,Country
count,84899,84325,92516
unique,3566,298,235
top,"Rock, Wisconsin, US",Texas,US
freq,28,5509,79299


In [13]:
print("the information on the data span from {} to {}".format(records.Date.min(),records.Date.max()))

the information on the data span from 2020-01-22 00:00:00 to 2020-04-18 00:00:00
